In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import meteva.method as mem
import numpy as np

***通过随机函数生成测试数据，用于后续检验函数调用示例***

In [2]:
ob = np.random.rand(2,20) * 10
fo = np.random.rand(2,20) * 10
ob_int = ob.astype(np.int8)
fo_int = fo.astype(np.int8)
grade_list = [3,5]

In [3]:
ob_int

array([[1, 3, 6, 9, 0, 5, 3, 3, 1, 2, 6, 2, 5, 0, 1, 2, 3, 3, 2, 5],
       [0, 5, 1, 7, 4, 3, 1, 9, 1, 5, 2, 6, 7, 6, 7, 3, 7, 4, 1, 3]],
      dtype=int8)

In [4]:
fo_int

array([[3, 5, 3, 6, 9, 1, 5, 8, 1, 6, 6, 1, 9, 2, 3, 2, 9, 6, 8, 4],
       [2, 7, 4, 0, 1, 5, 0, 6, 7, 4, 6, 6, 4, 3, 6, 5, 6, 8, 3, 8]],
      dtype=int8)

###  准确率  
**accuracy(ob, fo, grade_list=None)**  
统计观测和预报分类一致的样本数占比  

**参数说明：**  
 Ob:实况数据，任意维numpy数组   
 Fo:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 grade_list: 如果该参数为None，观测或预报值出现过的值都作为分类标记，如果该参数不为None，它必须是一个从小到大排列的实数，以其中列出的数值划分出的多个区间作为分类标签。对比预报和观测值不为整数的情况，grade_list 不能设置为None。  
 return:  0-1的实数，0代表无技巧，最优预报为1  
**调用示例：**  

In [5]:
mem.accuracy(ob_int,fo_int)

0.1

In [6]:
mem.accuracy(ob_int,fo_int,grade_list)

0.35

###  分类预报TS评分   
**ts_multi(ob, fo,grade_list=None)**  
基于原始数据计算针对每个类别的ts评分： Hit /(Hit + Misses+ False alarms) ，其中对于某个类别，Hit为预报值为该类别，且观测值也为该类别的样本数；Misses是预报值不为该类别，但观测值为该类别的样本数，False alarms为预报值为该类别，但观测值不为该类别的样本数。

**参数说明：**  
 Ob:实况数据，任意维numpy数组   
 Fo:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 grade_list:如果该参数为None，观测或预报值出现过的值都作为分类标记，如果该参数不为None，它必须是一个从小到大排列的实数，以其中列出的数值划分出的多个区间作为分类标签。对比预报和观测值不为整数的情况，grade_list 不能设置为None。   
 return: 返回一个一维numpy数组,每个元素为一个类别的ts评分。每个元素值为0到1的实数，完美预报对应值为1。 如果grade_list = None,即代表观测、预报为离散的变量，程序会提取所有出现过的值作为分类标记，返回数组的size也因此和分类标记的数量相同。如果grade_list != None则代表，预报和观测为连续变量，grade_list中必须为由小到大排列的实数，通过grade_list划分的区间来作为分类判据,此时返回数组的size 等于len(grade_list)+1。例如当grade_list = [3,5]，样本将被分为（负无穷,3),[3,5),[5,正无穷）三种类别。  
 
 **调用示例：**  

In [7]:
#grade_list = None,采用观测预报中出现过的0,1,2,3,4,5,6,7,8,9分别被作为类别标签，
mem.ts_multi(ob_int,fo_int)  

array([0.        , 0.1       , 0.14285714, 0.        , 0.        ,
       0.        , 0.18181818, 0.        , 0.        , 0.        ])

In [8]:
#grade_list = [3,5],将预报和观测分为（负无穷,3),[3,5),[5,正无穷）三种类别。
mem.ts_multi(ob_int,fo_int,grade_list)

array([0.33333333, 0.        , 0.27586207])

In [9]:
#分类ts评分算法的底层仍然是二分类预报的Ts评分算法，只是正负样本的判断标准的差异。
#在本程序库的二分类预报的ts评分函数中也可以接受grade_list参数，通过下面的例子可以了解其中差异
#在ts函数中，grade_list =[3,5]对应计算了两个评分值
#第一个评分代表将预报观测值分为（负无穷，3），[3,正无穷）
#第二个评分代表将预报观测值分为（负无穷，5），[5，正无穷）
#虽然grade_list和上一个例子中是一样的，但返回结果的size却少了1
mem.ts(ob_int,fo_int,grade_list)

array([0.64705882, 0.27586207])

###  分类预报ETS评分   
**ets_multi(ob, fo,grade_list=None)**  
基于原始数据计算针对每个类别的ets评分：(Hit-Hit_random) /(Hit + Misses+ False alarms - Hit_random) ，其中对于某个类别，Hit为预报值为该类别，且观测值也为该类别的样本数；Misses是预报值不为该类别，但观测值为该类别的样本数，False alarms为预报值为该类别，但观测值不为该类别的样本数。
Hit_randwom 为随机预报的命中数。   
  
**参数说明：**  
 Ob:实况数据，任意维numpy数组   
 Fo:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 grade_list:如果该参数为None，观测或预报值出现过的值都作为分类标记，如果该参数不为None，它必须是一个从小到大排列的实数，以其中列出的数值划分出的多个区间作为分类标签。对比预报和观测值不为整数的情况，grade_list 不能设置为None。   
 return: 返回一个一维numpy数组,每个元素为一个类别的ts评分。每个元素值为0到1的实数，完美预报对应值为1。 如果grade_list = None,即代表观测、预报为离散的变量，程序会提取所有出现过的值作为分类标记，返回数组的size也因此和分类标记的数量相同。如果grade_list != None则代表，预报和观测为连续变量，grade_list中必须为由小到大排列的实数，通过grade_list划分的区间来作为分类判据,此时返回数组的size 等于len(grade_list)+1。例如当grade_list = [3,5]，样本将被分为（负无穷,3),[3,5),[5,正无穷）三种类别。  
 
 
 ###  分类预报BIAS   
**bias_multi(ob, fo,grade_list=None)**  
基于原始数据计算针对每个类别的bias评分：(Hit + False alarms)/(Hit + Misses)，其中对于某个类别，Hit为预报值为该类别，且观测值也为该类别的样本数；Misses是预报值不为该类别，但观测值为该类别的样本数，False alarms为预报值为该类别，但观测值不为该类别的样本数。
  
  
**参数说明：**  
 Ob:实况数据，任意维numpy数组   
 Fo:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 grade_list:如果该参数为None，观测或预报值出现过的值都作为分类标记，如果该参数不为None，它必须是一个从小到大排列的实数，以其中列出的数值划分出的多个区间作为分类标签。对比预报和观测值不为整数的情况，grade_list 不能设置为None。   
 return: 返回一个一维numpy数组,每个元素为一个类别的ts评分。每个元素值为0到1的实数，完美预报对应值为1。 如果grade_list = None,即代表观测、预报为离散的变量，程序会提取所有出现过的值作为分类标记，返回数组的size也因此和分类标记的数量相同。如果grade_list != None则代表，预报和观测为连续变量，grade_list中必须为由小到大排列的实数，通过grade_list划分的区间来作为分类判据,此时返回数组的size 等于len(grade_list)+1。例如当grade_list = [3,5]，样本将被分为（负无穷,3),[3,5),[5,正无穷）三种类别。  
 
 
 
  
 ###  分类预报空报率   
**far_multi(ob, fo,grade_list=None)**  
基于原始数据计算针对每个类别的far评分： False alarms/(Hit + False alarms)，其中对于某个类别，Hit为预报值为该类别，且观测值也为该类别的样本数；Misses是预报值不为该类别，但观测值为该类别的样本数，False alarms为预报值为该类别，但观测值不为该类别的样本数。
  
  
**参数说明：**  
 Ob:实况数据，任意维numpy数组   
 Fo:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 grade_list:如果该参数为None，观测或预报值出现过的值都作为分类标记，如果该参数不为None，它必须是一个从小到大排列的实数，以其中列出的数值划分出的多个区间作为分类标签。对比预报和观测值不为整数的情况，grade_list 不能设置为None。   
 return: 返回一个一维numpy数组,每个元素为一个类别的ts评分。每个元素值为0到1的实数，完美预报对应值为1。 如果grade_list = None,即代表观测、预报为离散的变量，程序会提取所有出现过的值作为分类标记，返回数组的size也因此和分类标记的数量相同。如果grade_list != None则代表，预报和观测为连续变量，grade_list中必须为由小到大排列的实数，通过grade_list划分的区间来作为分类判据,此时返回数组的size 等于len(grade_list)+1。例如当grade_list = [3,5]，样本将被分为（负无穷,3),[3,5),[5,正无穷）三种类别。  
 
  ###  分类预报漏报率   
**mr_multi(ob, fo,grade_list=None)**  
基于原始数据计算针对每个类别的mr评分： Misses/(Hit + Misses)，其中对于某个类别，Hit为预报值为该类别，且观测值也为该类别的样本数；Misses是预报值不为该类别，但观测值为该类别的样本数，False alarms为预报值为该类别，但观测值不为该类别的样本数。


**参数说明：**  
 Ob:实况数据，任意维numpy数组   
 Fo:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 grade_list:如果该参数为None，观测或预报值出现过的值都作为分类标记，如果该参数不为None，它必须是一个从小到大排列的实数，以其中列出的数值划分出的多个区间作为分类标签。对比预报和观测值不为整数的情况，grade_list 不能设置为None。   
 return: 返回一个一维numpy数组,每个元素为一个类别的ts评分。每个元素值为0到1的实数，完美预报对应值为1。 如果grade_list = None,即代表观测、预报为离散的变量，程序会提取所有出现过的值作为分类标记，返回数组的size也因此和分类标记的数量相同。如果grade_list != None则代表，预报和观测为连续变量，grade_list中必须为由小到大排列的实数，通过grade_list划分的区间来作为分类判据,此时返回数组的size 等于len(grade_list)+1。例如当grade_list = [3,5]，样本将被分为（负无穷,3),[3,5),[5,正无穷）三种类别。  
 
 
 **调用示例：**  

In [10]:
mem.ets_multi(ob_int,fo_int,grade_list) #分类预报ETS评分

array([ 0.17948718, -0.13432836, -0.01204819])

In [11]:
mem.bias_multi(ob_int,fo_int,grade_list) #分类预报bias

array([0.6       , 0.9       , 1.46666667])

In [12]:
mem.far_multi(ob_int,fo_int,grade_list) #分类预报空报率

array([0.33333333, 1.        , 0.63636364])

In [13]:
mem.mr_multi(ob_int,fo_int,grade_list)  #分类预报漏报率

array([0.6       , 1.        , 0.46666667])

###  hss评分   
**hss(ob,fo,grade_list = None)**  
Heidke skill score，统计准确率相对于随机预报的技巧

**参数说明：**  
 Ob:实况数据，任意维numpy数组   
 Fo:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 grade_list: 如果该参数为None，观测或预报值出现过的值都作为分类标记，如果该参数不为None，它必须是一个从小到大排列的实数，以其中列出的数值划分出的多个区间作为分类标签。对比预报和观测值不为整数的情况，grade_list 不能设置为None。  
 return:  负无穷到1的实数，0代表无技巧，最优预报为1  
**调用示例：**  

In [14]:
mem.hss(ob_int,fo_int)

-0.004884856943475215

In [15]:
mem.hss(ob_int,fo_int,grade_list)

0.004784688995215294

###  hk评分   
**hk(ob,fo,grade_list = None)**  
Hanssen and Kuipers discriminant，统计准确率相对于随机预报的技巧

**参数说明：**  
 Ob:实况数据，任意维numpy数组   
 Fo:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 grade_list: 如果该参数为None，观测或预报值出现过的值都作为分类标记，如果该参数不为None，它必须是一个从小到大排列的实数，以其中列出的数值划分出的多个区间作为分类标签。对比预报和观测值不为整数的情况，grade_list 不能设置为None。  
 return:  负无穷到1的实数，0代表无技巧，最优预报为1  
**调用示例：**  

In [16]:
mem.hk(ob_int,fo_int)

-0.005043227665706041

In [17]:
mem.hk(ob_int,fo_int,grade_list)

0.004761904761904745

In [18]:
mem.hk(ob_int,fo_int,grade_list = [1.5,3.5])

-0.00575815738963528

In [19]:
mem.hk_yesorno(ob_int,fo_int,grade_list = [1.5,3.5])

array([0.06666667, 0.0971867 ])

In [20]:
mem.hss(ob_int,fo_int,grade_list = [1.5])

0.07692307692307666

In [21]:
mem.hss_yesorno(ob_int,fo_int,grade_list = [1.5])

array([0.07692308])

在以上示例中，观测和预报的数据都直接输入到评分函数中进行计算，然而有些情况下待检验的数据太大不能整体存入一个numpy数组中，或者不方便整体存入一个numpy数组中，此时就不能调用上面的方式调用评分函数。为此本程序库设计了一套可分块计算的检验程序。  
其检验步骤如下：  
***步骤1：根据需要将分块数据逐一输入到中间结果计算函数***  
***步骤2：将中间结果进行累加或合并***  
***步骤3：根据累加或合并的中间结果计算检验指标***  
通常上述计算中步骤1是最耗费计算资源，为了提高效率步骤1也可以采用**并行**的方式执行。此外，步骤1执行的结果也可**输出到文件**中，在后续的检验可以从中读入部分中间结果执行后续步骤，从而可以实现各种方式的分组检验，大大提高检验计算效率。

### 总样本数、正确样本数     
**tc(ob, fo, grade_list=None)**  
用来计算accuracy等检验指标的中间量  

**参数说明：**  
 Ob:实况数据，任意维numpy数组   
 Fo:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 grade_list: 如果该参数为None，观测或预报值出现过的值都作为分类标记，如果该参数不为None，它必须是一个从小到大排列的实数，以其中列出的数值划分出的多个区间作为分类标签。对比预报和观测值不为整数的情况，grade_list 不能设置为None。  
 return: 长度为2的一维numpy数组，其内容依次为总样本数、正确分类的样本数  
    
   
   
  
  
### 总样本数、正确样本数、 预报观测频率表    
**tcof(ob,fo,grade_list = None)**  
用来计算accuracy等检验指标的中间量  

**参数说明：**  
 **Ob**:实况数据，任意维numpy数组   
 **Fo**:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 **grade_list**: 一个从小到大排列的实数列表，以其中列出的数值划分出的多个区间作为分类标签，该参数不能设置为None。   
 **return**: shape为(len(grade_list)+2,2)的二维numpy数组,记为返回值为re，其中re[0,0] 为总样本数，re[0,1]为正确样本数据、re[1:,0]为观测频率表，re[1:,1]为预报频率表。  
   
   

### 命中、空报、漏报、正确否定     
**hfmc_multi(ob, fo, grade_list=None)**  
用来计算常用二分类预报检验指标的中间统计量    

**参数说明：**  
 Ob:实况数据，任意维numpy数组   
 Fo:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 grade_list:  如果该参数为None，观测或预报值出现过的值都作为分类标记，如果该参数不为None，它必须是一个从小到大排列的实数，以其中列出的数值划分出的多个区间作为分类标签。对比预报和观测值不为整数的情况，grade_list 不能设置为None。   
 return: 长度为N*4二numpy数组，当grade_list = None时，N等于观测预报中里的类别数， 当grade_list !=None时，N = len(grade_list) +1 , 每一列的内容依次为命中、空报、漏报、正确否定的样本数  
  

 **调用示例：**  

In [22]:
tc_array = np.zeros((2))
tcof_array = np.zeros((len(grade_list)+2,2))
hfmc_array = np.zeros((len(grade_list)+1,4))
for i in range(2):
    ob1 = ob_int[i,:]
    fo1 = fo_int[i,:]
    tc_array += mem.tc(ob,fo,grade_list)
    tcof_array += mem.tcof(ob,fo,grade_list)
    hfmc_array += mem.hfmc_multi(ob,fo,grade_list)
    

 **以下为根据合并后的中间统计量计算最终检验指标的函数：**
###  准确率  
**accuracy_tc(tc_array)**  
统计观测和预报分类一致的样本数占比  

**参数说明：**  
 tc_array:包含总样本数和正确样本数的多维数组，其中最后一维长度为2，分别记录了总样本数、正确样本数     
 return:  numpy数组，其中每个元素为0-1的实数，0代表无技巧，最优预报为1  
**调用示例：**  

In [23]:
mem.accuracy_tc(tc_array)

0.35

### 准确率   
**accuracy_tcof(tcof_array)**  
统计观测和预报分类一致的样本数占比

**参数说明：**   
 tcof_array:包含总样本数和正确样本数的多维数组，其中最后一维长度为2，其中tcof_array[...,0,0]记录了总样本数矩阵，其中tcof_array[...,0,1]记录了正确本数矩阵，tcof_array[...,1:,0]记录了观测频率表，tcof_array[...,1:,1]记录了预报频率表   
 return: numpy数组，其中每个元素为0-1的实数，0代表无技巧，最优预报为1  
**调用示例：**  

In [24]:
mem.accuracy_tcof(tcof_array)

0.35

### hss评分   
**hss_tcof(tcof_array)**  
Heidke skill score，统计准确率相对于随机预报的技巧

**参数说明：**   
 tcof_array:包含总样本数和正确样本数的多维数组，其中最后一维长度为2，其中tcof_array[...,0,0]记录了总样本数矩阵，其中tcof_array[...,0,1]记录了正确本数矩阵，tcof_array[...,1:,0]记录了观测频率表，tcof_array[...,1:,1]记录了预报频率表   
 return: numpy数组，其中每个元素为负无穷到1的实数，0代表无技巧，最优预报为1  
**调用示例：**  

In [25]:
mem.hss_tcof(tcof_array)

0.004784688995215294

### hk评分   
**hk_tcof(tcof_array)**  
Hanssen and Kuipers discriminant，统计准确率相对于随机预报的技巧

**参数说明：**   
 tcof_array:包含总样本数和正确样本数的多维数组，其中最后一维长度为2，其中tcof_array[...,0,0]记录了总样本数矩阵，其中tcof_array[...,0,1]记录了正确本数矩阵，tcof_array[...,1:,0]记录了观测频率表，tcof_array[...,1:,1]记录了预报频率表   
 return: numpy数组，其中每个元素为负无穷到1的实数，0代表无技巧，最优预报为1   
**调用示例：**  

In [26]:
mem.hk_tcof(tcof_array)

0.004761904761904745

基于hfmc计算分类预报的ts、ets、bias、far、mr等评分时，其函数调用方法和二分类模块中的方法是一样的。

In [27]:
mem.ts_hfmc(hfmc_array)

array([0.33333333, 0.        , 0.27586207])

In [28]:
mem.ets_hfmc(hfmc_array)

array([ 0.17948718, -0.13432836, -0.01204819])

In [29]:
mem.bias_hfmc(hfmc_array)

array([0.6       , 0.9       , 1.46666667])

In [30]:
mem.far_hfmc(hfmc_array)

array([0.33333333, 1.        , 0.63636364])

In [31]:
mem.mr_hfmc(hfmc_array)

array([0.6       , 1.        , 0.46666667])

在预报检验经常需要进行分组检验，获得不同类别预报的评分指标并进行对比。此时可以应用上述基于中间结果的检验函数对多维中间统计量的整体计算能力来简化代码的复杂度。  
**示例如下：**

In [32]:
day_count = 100
model_count = 3
ob = np.random.rand(day_count,10) * 10
fo = np.random.rand(model_count,day_count,10) * 10
ob_int = ob.astype(np.int8)
fo_int = fo.astype(np.int8)
tcof_array = np.zeros((model_count,len(grade_list)+2,2))
for i in range(day_count):
    ob1 = ob_int[i,:]
    for j in range(model_count):
        fo1 = fo_int[j,i,:]
        tcof_array[j,:,:] += mem.tcof(ob1,fo1,grade_list)

In [33]:
mem.accuracy_tcof(tcof_array)

array([0.347, 0.36 , 0.405])

In [34]:
mem.hss_tcof(tcof_array)

array([-0.03694576, -0.01825865,  0.05141793])

In [35]:
mem.hk_tcof(tcof_array)

array([-0.03653443, -0.01802068,  0.05064508])

以上只是展示了分类检验的维度为1的情况，实际上上述思路可以扩展至任意高维的情况。熟练使用中间统计量计算和合并方法，基于中间统计量整体计算分类问题下的检验指标数组，是提高代码编写效果的关键。上述检验函数的内部也都采用了numpy的整体计算方式实现，在计算效率上进行了最大程度的优化。